## Importando Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.linear_model import LinearRegression #importa a função de regressão linear do sklearn
from sklearn.metrics import mean_absolute_error #importa a metrica mean_absolute_error do sklearn
from sklearn.model_selection import train_test_split #importa o split

## Lendo o arquivo

In [ ]:
dados = pd.read_csv('mt_car.csv') #transforma o arquivo csv em um dataframe do pandas 

In [ ]:
dados.shape #para vermos quantas linhas e colunas o dataframe tem

In [134]:
dados.head(7) #para vermos as 7 linhas iniciais

,mpg,cyl,disp,drat,wt,qsec,vs,am,gear,carb,hp
0,21.0,6,160.0,3.90,2.620,16.46,0,1,4,4,110
1,21.0,6,160.0,3.90,2.875,17.02,0,1,4,4,110
2,22.8,4,108.0,3.85,2.320,18.61,1,1,4,1,93
3,21.4,6,258.0,3.08,3.215,19.44,1,0,3,1,110
4,18.7,8,360.0,3.15,3.440,17.02,0,0,3,2,175
5,18.1,6,225.0,2.76,3.460,20.22,1,0,3,1,105
6,14.3,8,360.0,3.21,3.570,15.84,0,0,3,4,245


In [ ]:
dados.info()

In [ ]:
#exclui a coluna que contém os nomes dos carros, porque não será necessário
#onde axis = 1 para coluna e 0 para linha
dados = dados.drop(['Unnamed: 0'], axis = 1)

In [ ]:
#Movendo a coluna hp para o final do dataframe

#a função dados.columns retorna o nome das colunas
#e a função to_list() transforma o retorno da função de cima em lista
#e armazena tudo em colunas
colunas = dados.columns.to_list()

#eu copio e colo a saída da função de cima^ na variavel debaixo
nova_coluna = ['mpg', 'cyl', 'disp', 'drat', 'wt', 'qsec', 'vs', 'am', 'gear', 'carb', 'hp']

#e então transformo o dataframe
dados = dados[nova_coluna]

#verificando alteração
dados.head()

## Correlação

In [ ]:
#Correlograma
#usando a função dados.corr() para gerar um dataframe com as correlações
#e a biblioteca seaborn para gera um mapa de calor com a função heatmap 
plt.figure(figsize=(12,6))
sns.heatmap(dados.corr(),annot=True,cmap='Blues')
#salvando e imprimindo o correlograma 
plt.savefig("correlograma_irado")
plt.show()

In [ ]:
#mostrando a correlação da variavel que quero prever com mais casas decimais
#deve se notar que a correlação de hp com hp é sempre 1 pois eles tem a mesma taxa de variação!
#isso sempre ocorre entre duas variaveis iguais!!!
dados.corr()['hp']

In [ ]:
#demonstrando correlações mais fortes sem distinção de negativa ou positiva
abs(dados.corr()['hp'])>=0.75
#a função abs() retorna os valores absolutos das correlações

## Atributos

In [ ]:
#atributos que irei estudar: 
#           mpg: miles per galon, ou milhas por galão(metrica imperial para volume)
#           cyl: número de cilindros
#           disp: é a quantidade de ar que é deslocado para dentro do motor
#           hp: quantos cavalos o carro tem 
#           drat: relação do eixo traseiro
#           wt: peso do carro em libras dividido por 1000
#           qsec: quanto tempo o carro percorre 1/4 de milha     
#           vs: tipo de motor(0 = em v; 1 = em linha reta)
#           am: tipo de transmissão(0 = automatico; 1 = manual)
#           gear: número de marchas sem incluir a ré
#           carb: número de carburadores

## Métricas

In [ ]:
def metricas(dado_real, previsao):
    
#MAE(Mean Absolute Error) é soma das diferença absolutas(sem diferença de sinal, positivo ou negativo) entre os valores reais 
#e  o valor da previsão
    MAE = mean_absolute_error(dado_real, previsao)

#RMSE(Root Mean Square Error) é o desvio padrão dos resíduos(a distância entre os pontos da previsão e os pontos reais(y_test)).
#ou seja, mostra o quão próximos os dados da previsão estão dos dados reais
    RMSE = np.sqrt(mean_absolute_error(dado_real, previsao))
    
    
#imprime na tela o RMSE(Root Mean Square Error) e o MAE
    print("RMSE: {} \nMAE: {}\n".format(RMSE, MAE))
#Quanto menor o valor das duas métricas melhor, porém deve-se tomar cuidado com o overfitting do modelo!


#Comparando os dados da previsão com os dados reais de teste

#a função round serve para arredondar os números de uma matriz numpy
#arredonda os número e deixa uma casa decimal
    arredondado = previsao.round(1)
    print("y_teste:  {} \nprevisões:{}".format(dado_real, arredondado))

## Split treino e teste

In [ ]:
#revendo o formato do dataframe
dados.head()

In [ ]:
#experimento 1, separando todas as variaveis para treinar o modelo
X = dados.iloc[:, 0:9].values #os atributos preditores
y = dados.iloc[:, 10].values #o hp que será previsto

#a função iloc funciona buscando linhas ou colunas de um dataframe a partir de seus índices
#e a função loc tem o mesmo objetivo do iloc, só que busca a partir dos labels das linhas ou colunas

In [ ]:
#separa dados de treino e de teste aleotoriamente com uma proporção de 90/10, treinamento/teste
#e com random_state=123, uma seed é definida para que o experimento possa ser replicado
#e retorna uma matriz ou array  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)


X_train.shape, X_test.shape, y_train.shape, y_test.shape# sempre olhar o formato do split!

## Treinando o modelo

In [ ]:
#a função de regressão linear do sklearn 
modelo1 = LinearRegression()
modelo1.fit(X_train, y_train)#treinando o modelo com os dados respectivos de treinamento

previsoes = modelo1.predict(X_test)#prevê o hp(y) baseado nos dados de X_test

## Comparando os dados da previsão com os dados reais de teste

In [ ]:
metricas(y_test, previsoes)

In [ ]:
#nota-se que as previsões foram boas, mas podem melhorar!

## Melhorando o modelo a partir das correlações mais fortes

In [127]:
abs(dados.corr()['hp'])>=0.7

mpg      True
cyl      True
disp     True
drat    False
wt      False
qsec     True
vs       True
am      False
gear    False
carb     True
hp       True
Name: hp, dtype: bool

In [128]:
#experimento 2
#separando todas as variaveis para treinar o modelo
variaveis = ['mpg', 'cyl', 'disp', 'carb', 'qsec', 'vs']
X2 = dados[variaveis] #os atributos preditores

y2 = dados.iloc[:, 10].values #o hp que será previsto

X2.head()

,mpg,cyl,disp,carb,qsec,vs
0,21.0,6,160.0,4,16.46,0
1,21.0,6,160.0,4,17.02,0
2,22.8,4,108.0,1,18.61,1
3,21.4,6,258.0,1,19.44,1
4,18.7,8,360.0,2,17.02,0


In [129]:
X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.1, random_state=123)


X_train.shape, X_test.shape, y_train.shape, y_test.shape

((28, 6), (4, 6), (28,), (4,))

## Treinando o modelo com as novas variáveis

In [135]:
modelo2 = LinearRegression()
modelo2.fit(X_train, y_train)

previsoes = modelo2.predict(X_test)

## Comparando os dados da previsão com os dados reais de teste

In [136]:
metricas(y_test, previsoes)

RMSE: 3.3047164910370106 
MAE: 10.921151086131971

y_teste:  [ 62 109 105  91] 
previsões:[ 90.2  97.5 105.3  87.4]


## Ajuda e algumas dicas sobre o jupyter labs ou jupyter notebook

use "?train_test_split" para obter uma ajuda sobre algum elemento

use "!" para usar o terminal 

para criar uma nova célula abaixo da atual, aperte ESC+B

para apagar uma célula aperte ESC e aperte D duas vezes

para criar uma célula markdown, aperte ESC+M

para cria uma célula de título, transforme em markdown, depois "## o seu texto"